## Loading the data

In [1]:
import numpy as np
import os
from sklearn.impute import SimpleImputer
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
import time
import xgboost as xgb
import warnings 
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# giving directory name
fitbit_data_dirs = ['/content/drive/MyDrive/Data/pilot/fitbit/control','/content/drive/MyDrive/Data/pilot/fitbit/intervention']
self_obs_data_dir = '/content/drive/MyDrive/Data/pilot/old/selfObservation'
questionnaire_data_dir = '/content/drive/MyDrive/Data/pilot/old/questionnaire'

In [4]:
ext = ('.csv')

In [5]:
# Read fitbit_data
fitbit_data_lst = []
for fitbit_data_dir in fitbit_data_dirs:
  for files in os.listdir(fitbit_data_dir):
      if files.endswith(ext):
          #print(fitbit_data_dir + "/" + files)
          fitbit_data = pd.read_csv(fitbit_data_dir + "/" + files,delimiter=',', dtype=object)
          fitbit_data['pid'] = files[0:-4]
          fitbit_data.rename(columns={'date': 'timestamp'}, inplace=True)
          fitbit_data_lst.append(fitbit_data)

fitbit_data = pd.concat(fitbit_data_lst, axis=0, ignore_index=True)

In [6]:
# fitbit_data - Keep only useful columns
fitbit_data = fitbit_data[["pid",'timestamp',"sleepAsleepMinutes","distance", 'steps']]
int_columns = ["sleepAsleepMinutes","distance", 'steps']
fitbit_data.head()

,pid,timestamp,sleepAsleepMinutes,distance,steps
0,613b0c716a9a5d196c5cc6f0,2021-09-16 01:00:00,NaN,NaN,NaN
1,613b0c716a9a5d196c5cc6f0,2021-09-16 01:01:00,NaN,NaN,NaN
2,613b0c716a9a5d196c5cc6f0,2021-09-16 01:02:00,NaN,NaN,NaN
3,613b0c716a9a5d196c5cc6f0,2021-09-16 01:03:00,NaN,NaN,NaN
4,613b0c716a9a5d196c5cc6f0,2021-09-16 01:04:00,NaN,NaN,NaN


In [7]:
for i in int_columns:
  fitbit_data[i] = fitbit_data[i].astype(str).str.replace(',','.')
  fitbit_data[i] = fitbit_data[i].astype(str).str.replace('<','')
  fitbit_data[i] = fitbit_data[i].astype(str).str.replace('Ο','0')
  fitbit_data[i] = fitbit_data[i].astype(str).str.replace(' ','')
  fitbit_data[i] = fitbit_data[i].replace('nan',np.nan)
  fitbit_data[i] = fitbit_data[i].replace('-',np.nan)

fitbit_data = fitbit_data.fillna(0)

In [8]:
#Convert columns to their correct type
fitbit_data['timestamp'] = pd.to_datetime(fitbit_data['timestamp']).dt.date
fitbit_data['sleepAsleepMinutes'] = fitbit_data['sleepAsleepMinutes'].astype("int")
fitbit_data['distance'] = fitbit_data['distance'].astype("int")
fitbit_data['steps'] =  fitbit_data['steps'].astype("int")

In [9]:
fitbit_data.head()

,pid,timestamp,sleepAsleepMinutes,distance,steps
0,613b0c716a9a5d196c5cc6f0,2021-09-16,0,0,0
1,613b0c716a9a5d196c5cc6f0,2021-09-16,0,0,0
2,613b0c716a9a5d196c5cc6f0,2021-09-16,0,0,0
3,613b0c716a9a5d196c5cc6f0,2021-09-16,0,0,0
4,613b0c716a9a5d196c5cc6f0,2021-09-16,0,0,0


In [10]:
# Group by 
fitbit_data = fitbit_data.groupby(by=["pid","timestamp"], as_index=False, sort=False).sum()
fitbit_data.head()

,pid,timestamp,sleepAsleepMinutes,distance,steps
0,613b0c716a9a5d196c5cc6f0,2021-09-16,0,0,0
1,613b0c716a9a5d196c5cc6f0,2021-09-17,0,2200,35
2,613b0c716a9a5d196c5cc6f0,2021-09-18,0,0,0
3,613b0c716a9a5d196c5cc6f0,2021-09-19,0,0,0
4,613b0c716a9a5d196c5cc6f0,2021-09-20,0,0,0


### Fitbit adherence

In [11]:
print(fitbit_data.columns)
pids = list(fitbit_data["pid"].unique())

for pid in pids:
  temp = fitbit_data.loc[fitbit_data['pid'] == pid] 
  number_of_days = len(temp["timestamp"].unique())
  no_exercise_days_count = len(temp["steps"].loc[lambda x : x!=0])
  print(pid,no_exercise_days_count,number_of_days,str((no_exercise_days_count/number_of_days) * 100) + "%")

Index(['pid', 'timestamp', 'sleepAsleepMinutes', 'distance', 'steps'], dtype='object')
613b0c716a9a5d196c5cc6f0 2 100 2.0%
613b11756a9a5d196c5cc6f2 58 107 54.20560747663551%
613b2a7b6a9a5d196c5cc6f6 89 107 83.17757009345794%
613f1e0e6a9a5d196c5cc6fc 101 102 99.01960784313727%
613f3b066a9a5d196c5cc6fe 88 104 84.61538461538461%
614037d86a9a5d196c5cc700 68 103 66.01941747572816%
6141b83b6a9a5d196c5cc702 96 102 94.11764705882352%
615bfda96a9a5d196c5cc70e 37 82 45.1219512195122%
615d5af76a9a5d196c5cc710 46 81 56.79012345679012%
615ff17b6a9a5d196c5cc712 24 79 30.37974683544304%
618a36106a9a5d196c5cc724 13 109 11.926605504587156%
61935f406a9a5d196c5cc72a 0 102 0.0%
619364876a9a5d196c5cc72c 33 102 32.35294117647059%
61a483586a9a5d196c5cc730 13 89 14.606741573033707%
61ea9650af99b3594fb760ca 0 89 0.0%
61eab033af99b3594fb760cc 31 85 36.470588235294116%
61eab41baf99b3594fb760ce 83 86 96.51162790697676%
61c194936a9a5d196c5cc738 114 131 87.02290076335878%
61c195f56a9a5d196c5cc73a 18 124 14.51612903

In [12]:
# Read self_obs_data
self_obs_data_lst = []
for files in os.listdir(self_obs_data_dir):
    if files.endswith(ext):
        #print(files)
        self_obs_data = pd.read_csv(self_obs_data_dir + "/" + files,delimiter=';')
        self_obs_data['pid'] = files[0:-4]
        self_obs_data_lst.append(self_obs_data)

self_obs_data = pd.concat(self_obs_data_lst, axis=0, ignore_index=True)

In [13]:
score_name_dict = {"mother": "score_mother", 
                   "foodpractices": "score_foodpractices",
                   "strengthsdifficulties": "score_strengthsdifficulties",
                   "psycho": "score_psycho", 
                   "post_mother": "score_post_mother" , 
                   "post_foodpractices": "score_post_foodpractices",
                   "post_strengthsdifficulties": "score_post_strengthsdifficulties"
}

# Read questionaire data
questionnaire_data_lst = []
file_names = []
for files in os.listdir(questionnaire_data_dir):
    if files.endswith(ext):        
        file_names.append(files)
        questionnaire_data = pd.read_csv(questionnaire_data_dir + "/" + files,delimiter=';')
        questionnaire_data.columns = questionnaire_data.columns.str.replace("\n", " ")
        questionnaire_data_lst.append(questionnaire_data)

questionnaire_data = questionnaire_data_lst[0]
for col in questionnaire_data.columns:
    if '- t' in col:
        del questionnaire_data [col]

for i in range(1,len(questionnaire_data_lst)):
  for col in questionnaire_data_lst[i].columns:
    if '- t' in col:
        del questionnaire_data_lst[i] [col]
  questionnaire_data = pd.merge(questionnaire_data, questionnaire_data_lst[i], how='outer', left_on=['pid'], right_on = ['pid'])

In [14]:
questionnaire_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 0 to 14
Columns: 390 entries, pid to Φέρνει σε πέρας μία εργασία, έχει καλή προσοχή - v_y
dtypes: float64(167), int64(176), object(47)
memory usage: 45.8+ KB


In [15]:
print(fitbit_data.head())

                        pid   timestamp  sleepAsleepMinutes  distance  steps
0  613b0c716a9a5d196c5cc6f0  2021-09-16                   0         0      0
1  613b0c716a9a5d196c5cc6f0  2021-09-17                   0      2200     35
2  613b0c716a9a5d196c5cc6f0  2021-09-18                   0         0      0
3  613b0c716a9a5d196c5cc6f0  2021-09-19                   0         0      0
4  613b0c716a9a5d196c5cc6f0  2021-09-20                   0         0      0


In [16]:
print(self_obs_data.head())

             timestamp                 time target  \
0  2022-02-25 04:00:26  1 ώρα ή περισσότερο    Ναι   
1  2022-02-21 04:00:28                  NaN    NaN   
2  2022-03-10 04:00:32                  NaN    NaN   
3  2022-03-10 04:00:33                  NaN    NaN   
4  2022-02-28 04:00:17                  NaN    Ναι   

                                                desc  \
0                                  Προπόνηση μπάσκετ   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  Από δ. πλάνο: \nπριν το σχολείο: \n1 ποτήρι γά...   

                        pid  weight  eat eatJunk home junk descFruit delivery  \
0  62125bdb2a666e100124a50d     NaN  NaN     NaN  NaN  NaN       NaN      NaN   
1  620241ce2a666e100124a507    40.0  NaN     NaN  NaN  NaN       NaN      NaN   
2  620241ce2a666e100124a507    41.0  NaN     NaN  NaN  NaN       NaN      NaN   
3  620241ce2a6

In [17]:
print(questionnaire_data.head())

                        pid  ΙΚΤΕΡΟΣ - v  ΥΠΟΓΛΥΚΑΙΜΙΑ - v  \
0  61ea9650af99b3594fb760ca            0                 0   
1  61eab033af99b3594fb760cc            0                 0   
2  61eab41baf99b3594fb760ce            0                 0   
3  61c192f96a9a5d196c5cc734            0                 0   
4  61c193b26a9a5d196c5cc736            0                 0   

   ΠΑΡΑΜΟΝΗ ΣΕ ΜΕΝ / ΜΑΦ - v ΛΟΙΠΑ ΠΡΟΒΛΗΜΑΤΑ - v Αίτιο - v  \
0                          0                    -       NaN   
1                          0                    -       NaN   
2                          0                    -       NaN   
3                          0                    -       NaN   
4                          0                    -       NaN   

   ΗΜΕΡΕΣ ΠΑΡΑΜΟΝΗΣ - v HbA1c - v  Chol - v  HDL - v  ...  \
0                   NaN         -       139       62  ...   
1                   NaN       5,2       185       87  ...   
2                   NaN       5,2       137       60  ...   
3   

### Remove useless columns to allow join

In [18]:
# Print sub-data
print(fitbit_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2571 entries, 0 to 2570
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pid                 2571 non-null   object
 1   timestamp           2571 non-null   object
 2   sleepAsleepMinutes  2571 non-null   int64 
 3   distance            2571 non-null   int64 
 4   steps               2571 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 100.6+ KB
None


In [19]:
print(self_obs_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  431 non-null    object 
 1   time       74 non-null     object 
 2   target     395 non-null    object 
 3   desc       297 non-null    object 
 4   pid        431 non-null    object 
 5   weight     74 non-null     float64
 6   eat        80 non-null     object 
 7   eatJunk    84 non-null     object 
 8   home       235 non-null    object 
 9   junk       124 non-null    object 
 10  descFruit  72 non-null     object 
 11  delivery   111 non-null    object 
 12  descMilk   37 non-null     object 
 13  comments   24 non-null     object 
 14  descSalad  16 non-null     object 
dtypes: float64(1), object(14)
memory usage: 50.6+ KB
None


In [20]:
print(questionnaire_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 0 to 14
Columns: 390 entries, pid to Φέρνει σε πέρας μία εργασία, έχει καλή προσοχή - v_y
dtypes: float64(167), int64(176), object(47)
memory usage: 45.8+ KB
None


In [21]:
# self_obs_data
self_obs_data = self_obs_data[["pid",'timestamp', 'target',"weight","home","junk"]]

In [22]:
for i in ['target',"weight","home","junk"]:
  self_obs_data[i] = self_obs_data[i].astype(str).str.replace(',','.')
  self_obs_data[i] = self_obs_data[i].astype(str).str.replace('<','')
  self_obs_data[i] = self_obs_data[i].astype(str).str.replace('Ο','0')
  self_obs_data[i] = self_obs_data[i].astype(str).str.replace(' ','')
  self_obs_data[i] = self_obs_data[i].replace('nan',np.nan)
  self_obs_data[i] = self_obs_data[i].replace('-',np.nan)

In [23]:
remap = {'Ναι': 1, 'Όχι': 0, ' ':np.nan}
self_obs_data['target'] = self_obs_data['target'].map(remap)
self_obs_data['home'] = self_obs_data["home"].map(remap)
self_obs_data['junk'] = self_obs_data["junk"].map(remap)
self_obs_data['timestamp'] = pd.to_datetime(self_obs_data['timestamp']).dt.date

self_obs_data.head()

,pid,timestamp,target,weight,home,junk
0,62125bdb2a666e100124a50d,2022-02-25,1.0,NaN,NaN,NaN
1,620241ce2a666e100124a507,2022-02-21,NaN,40.0,NaN,NaN
2,620241ce2a666e100124a507,2022-03-10,NaN,41.0,NaN,NaN
3,620241ce2a666e100124a507,2022-03-10,NaN,41.0,NaN,NaN
4,620241ce2a666e100124a507,2022-02-28,1.0,NaN,NaN,NaN


### Self-observation adherence

In [24]:
for pid in pids:
  temp = fitbit_data.loc[fitbit_data['pid'] == pid] 
  number_of_days = len(temp["timestamp"].unique())

  temp_obs = self_obs_data.loc[self_obs_data['pid'] == pid] 
  seb_obs_rec = len(temp_obs["pid"])

  print(pid,seb_obs_rec,number_of_days,str((seb_obs_rec/number_of_days) * 100) + "%")

613b0c716a9a5d196c5cc6f0 0 100 0.0%
613b11756a9a5d196c5cc6f2 0 107 0.0%
613b2a7b6a9a5d196c5cc6f6 0 107 0.0%
613f1e0e6a9a5d196c5cc6fc 0 102 0.0%
613f3b066a9a5d196c5cc6fe 0 104 0.0%
614037d86a9a5d196c5cc700 0 103 0.0%
6141b83b6a9a5d196c5cc702 0 102 0.0%
615bfda96a9a5d196c5cc70e 0 82 0.0%
615d5af76a9a5d196c5cc710 0 81 0.0%
615ff17b6a9a5d196c5cc712 0 79 0.0%
618a36106a9a5d196c5cc724 0 109 0.0%
61935f406a9a5d196c5cc72a 0 102 0.0%
619364876a9a5d196c5cc72c 0 102 0.0%
61a483586a9a5d196c5cc730 0 89 0.0%
61ea9650af99b3594fb760ca 79 89 88.76404494382022%
61eab033af99b3594fb760cc 35 85 41.17647058823529%
61eab41baf99b3594fb760ce 7 86 8.13953488372093%
61c194936a9a5d196c5cc738 7 131 5.343511450381679%
61c195f56a9a5d196c5cc73a 6 124 4.838709677419355%
61d6cc7a5828a5043783a1f1 5 104 4.807692307692308%
61d6cd965828a5043783a1f3 76 103 73.7864077669903%
61d6ce215828a5043783a1f5 80 102 78.43137254901961%
61d6cf035828a5043783a1f7 39 86 45.348837209302324%
61dd93a2af99b3594fb760c4 6 100 6.0%
620229f52a666e

### Join them

In [25]:
# Join data based on pid and timestamp
temp_data = pd.merge(fitbit_data, self_obs_data,  how='left', left_on=['pid',"timestamp"], right_on = ['pid',"timestamp"])
del fitbit_data
del self_obs_data
temp_data.head()

,pid,timestamp,sleepAsleepMinutes,distance,steps,target,weight,home,junk
0,613b0c716a9a5d196c5cc6f0,2021-09-16,0,0,0,NaN,NaN,NaN,NaN
1,613b0c716a9a5d196c5cc6f0,2021-09-17,0,2200,35,NaN,NaN,NaN,NaN
2,613b0c716a9a5d196c5cc6f0,2021-09-18,0,0,0,NaN,NaN,NaN,NaN
3,613b0c716a9a5d196c5cc6f0,2021-09-19,0,0,0,NaN,NaN,NaN,NaN
4,613b0c716a9a5d196c5cc6f0,2021-09-20,0,0,0,NaN,NaN,NaN,NaN


In [26]:
# Remove lines that have steps = NAN
temp_data = temp_data[temp_data['steps'].notna()]
# Remove columns that are all empty
temp_data = temp_data.dropna(axis=1, how='all')

In [27]:
# Join data based on pid
data = pd.merge(temp_data, questionnaire_data,  how='left', left_on=['pid'], right_on = ['pid'])
del temp_data

### Delete Columns

In [28]:
#delete useless columns

columns_to_drop = ['Κ.Τ ΛΟΓΩ - v', 'Αίτιο - v', 'ΗΜΕΡΕΣ ΠΑΡΑΜΟΝΗΣ - v', 'Άλλα νοσήματα - v', 'Είδος Αλλεργίας - v', 'Aλλα αίτια νοσηλείας - v',\
                          'Είδος ΦΑΡΜΑΚΑ / ΒΙΤΑΜΙΝΕΣ / ΣΥΜΠΛΗΡΩΜΑΤΑ - v', 'Εάν έφαγε ψάρι ή θαλασσινό το παιδί σας τον προηγούμενο μήνα προσδιορίστε το είδος: - v',\
                          'Εφόσον υπήρξαν δυσκολίες προσδιορίστε το είδος: - v', 'Εάν έφαγε ψάρι ή θαλασσινό το παιδί σας την προηγούμενη βδομάδα προσδιορίστε το είδος: - v',\
                          'Παρακαλούμε διευκρινίστε την ασθένεια - v', 'ΗΜΕΡΟΜΗΝΙΑ ΓΕΝΝΗΣΗΣ - v', 'ΗΜΕΡΟΜΗΝΙΑ ΚΛΙΝΙΚΗΣ ΕΞΕΤΑΣΗΣ - v', 'ΑΔΕΛΦΙΑ: ΗΛΙΚΙΕΣ ΚΑΙ ΦΥΛΟ - v',\
                          'ΕΚΠΑΙΔΕΥΣΗ ΜΗΤΡΟΣ - v', 'ΕΚΠΑΙΔΕΥΣΗ ΠΑΤΡΟΣ - v', 'ΕΠΑΓΓΕΛΜΑ ΜΗΤΡΟΣ - v', 'ΕΠΑΓΓΕΛΜΑ ΠΑΤΡΟΣ - v', 'ΛΟΙΠΑ ΠΡΟΒΛΗΜΑΤΑ - v', 'ΑΔΕΛΦΙΑ: ΗΛΙΚΙΕΣ ΚΑΙ ΦΥΛΟ - v','Εάν το παιδί σας καταναλώνει επεξεργασμένο κρέας προσδιορίστε το είδος: - v', 'Μικρό ενδιαφέρον ή λίγη απόλαυση στις δραστηριότητές μου - v','Νιώθετε καταβεβλημένος(η), κατατεθλιμμένος(η) ή απελπισμένος(η) - v', 'Εάν επιλέξατε κάποια προβλήματα, πόση δυσκολία προκάλεσαν τα προβλήματα αυτά στη δουλειά σας, στις οικιακές εργασίες σας ή στην επικοινωνία σας με άλλα άτομα; - v',\
                          '. Έχετε πρόβλημα να αποκοιμηθείτε ή να συνεχίσετε τον ύπνο σας ή κοιμάστε υπερβολικά - v', 'Νιώθετε κουρασμένος(η) ή έχετε λίγη ενέργεια - v', 'Έχετε λίγη όρεξη ή τρώτε υπερβολικά - v','Νιώθετε άσχημα για τον εαυτό σας ή ότι έχετε αποτύχει ή ότι έχετε απογοητεύσει τον εαυτό σας ή την οικογένειά σας - v',\
                          'Έχετε πρόβλημα συγκέντρωσης σε κάποιες ενέργειες, όπως όταν διαβάζετε την εφημερίδα ή όταν παρακολουθείτε τηλεόραση - v',\
            'Κινείστε ή μιλάτε τόσο αργά που άλλοι άνθρωποι θα το παρατηρούσαν Ή το αντίθετο – είστε τόσο ανήσυχος(η) ή νευρικός(ή), που κινείστε πολύ περισσότερο από το συνηθισμένο - v',\
            'Σκεπτόσαστε ότι θα ήταν καλύτερα αν είχατε πεθάνει ή σκεπτόσαστε να προκαλέσετε κακό στον εαυτό σας με κάποιο τρόπο - v']

data = data.drop(columns=columns_to_drop)


In [29]:
#delete medial clinical columns
data = data.drop(columns=['ΥΨΟΣ - v', 'Βάρος - v', 'ΠΕΡΙΜΕΤΡΟΣ ΜΕΣΗΣ - v', 'ΑΡΤΗΡΙΑΚΗ ΠΙΕΣΗ ΣΥΣΤΟΛΙΚΗ - v', 'ΑΡΤΗΡΙΑΚΗ ΠΙΕΣΗ ΔΙΑΣΤΟΛΙΚΗ - v',\
                  'Tanner Pubis - v', 'Tanner μασχάλη - v', 'Θυρεοειδής - v', 'Όγκος όρχεων Δ - v', 'Μελανίζουσα ακάνθωση - v', 'Mήκος πέους - v',\
                  'Όγκος όρχεων Αρ - v', 'Ειδικά Χαρακτηριστικά - v'])                

In [30]:
#delete mother columns(only need score) pre
data =  data.drop(columns=['Μαλώνω και φωνάζω στο παιδί μου όταν φέρεται άσχημα. - v_y', 'Δείχνω στο παιδί μου κατανόηση όταν είναι αναστατωμένο. - v_y',\
                           'Υποκύπτω στις επιθυμίες του παιδιού μου όταν επιμένει έντονα σε αυτές. - v_y', 'Επαινώ το παιδί μου όταν είναι καλό και υπάκουο. - v_y',\
                           'Απειλώ περισσότερο το παιδί μου με τιμωρία παρά το τιμωρώ. - v_y', 'Ξεσπάω το θυμό μου στο παιδί μου. - v_y', 'Δέρνω το παιδί μου όταν δεν υπακούει. - v_y',\
                           'Λαμβάνω υπόψη μου τις επιθυμίες του παιδιού μου όταν κάνω οικογενειακά σχέδια. - v_y', 'Επιτρέπω στο παιδί μου να πει τη γνώμη του για τους κανόνες που υπάρχουν στην οικογένεια. - v_y',\
                           'Λέω στο παιδί μου με ποιο τρόπο θα το τιμωρήσω, αλλά δεν εφαρμόζω την τιμωρία. - v_y', 'Κατανοώ τα συναισθήματα του παιδιού μου. - v_y', 'Χρησιμοποιώ την τιμωρία ως μέσο πειθαρχίας. - v_y',\
                           'Λαμβάνω υπόψη μου τι θέλει το παιδί μου, πριν του ζητήσω να κάνει κάτι. - v_y', 'Λέω στο παιδί μου πως νοιώθω, όταν συμπεριφέρεται καλά ή όταν συμπεριφέρεται άσχημα. - v_y',\
                           'Παροτρύνω το παιδί μου να συζητά μαζί μου τα προβλήματά του. - v_y', 'Πιστεύω ότι είναι δύσκολο να μάθω στο παιδί μου να πειθαρχεί. - v_y', 'Ζητώ από το παιδί μου να εκφράζει τη γνώμη του ακόμα και όταν διαφωνεί μαζί μου. - v_y',\
                           'Τιμωρώ το παιδί μου απαγορεύοντάς του να κάνει κάτι που του αρέσει χωρίς να του δίνω εξηγήσεις. - v_y', 'Λέω στο παιδί μου πόσο σημαντικό είναι να ακολουθεί τους κανόνες της οικογένειας. - v_y',\
                           'Κάνω συστάσεις στο παιδί μου για να βελτιωθεί η συμπεριφορά του - v_y', 'Εξηγώ στο παιδί μου γιατί πρέπει να τηρούνται οι κανόνες της οικογένειας - v_y', 'Κακομαθαίνω το παιδί μου - v_y',\
                           'Δείχνω τρυφερότητα στο παιδί μου - v_y', 'Χρησιμοποιώ απειλές για να τιμωρήσω το παιδί μου χωρίς να του δίνω εξηγήσεις - v_y', 'Βοηθώ το παιδί μου να καταλάβει τις συνέπειες μιας κακής πράξης του μέσα από διάλογο και συζήτηση - v_y',\
                           'Τιμωρώ το παιδί μου στέλνοντάς το στο δωμάτιό του χωρίς να του δίνω εξηγήσεις - v_y', 'Συζητώ με το παιδί μου για τις συνέπειες των πράξεών του - v_y', 'Ασκώ κριτική στο παιδί μου όταν η συμπεριφορά του δεν ανταποκρίνεται στις προσδοκίες μου - v_y',\
                           'Χαστουκίζω το παιδί μου όταν δεν συμπεριφέρεται σωστά - v_y'])

In [31]:
#delete strengthdiff columns(only need score) pre
data =  data.drop(columns=['Έχει τουλάχιστον ένα φίλο - v_y', 'Συνεχώς στριφογυρίζει νευρικά ή δεν στέκεται ήσυχος/η, έχει νευρικότητα - v_y',
                           'Συχνά είναι δυστυχισμένος/η, αποκαρδιωμένος/η ή κλαίει - v_y', 'Συχνά μαλώνει με τα άλλα παιδιά ή τα κοροϊδεύει, τα φοβερίζει, τα κτυπά - v_y',\
                           'Η προσοχή του/της διασπάται εύκολα, δυσκολεύεται να συγκεντρωθεί - v_y', 'Γενικά είναι συμπαθής στα άλλα παιδιά - v_y',\
                           'Είναι καλός/ή με τα μικρότερα παιδιά - v_y', 'Σε καινούριες καταστάσεις είναι νευρικός/η ή δείχνει συμπεριφορά προσκόλλησης, εύκολα χάνει την αυτοπεποίθησή του/της - v_y',\
                           'Τα άλλα παιδιά τον/την έχουν στο μάτι ή τον/την κοροϊδεύουν, τον/την φοβερίζουν, τον/την χτυπούν - v_y', 'Συχνά λέει ψέμματα ή εξαπατά - v_y',\
                           'Λαμβάνει υπόψη τα συναισθήματα των άλλων - v_y', 'Ανήσυχος/η και υπερδραστήριος/α, δεν μπορεί να παραμείνει ήρεμος/η, ακίνητος/η για πολύ ώρα - v_y',\
                           'Παραπονείται συχνά για πονοκεφάλους, πόνους στο στομάχι ή αδιαθεσία - v_y', 'Μοιράζεται εύκολα με τα άλλα παιδιά (κεράσματα, παιχνίδια, μολύβια κτλ) - v_y',\
                           'Συχνά έχει ξεσπάσματα νεύρων ή είναι ευέξαπτος - v_y', 'Μάλλον μοναχικός/η, τείνει να παίζει μόνος/η - v_y',\
                           'Γενικά είναι υπάκουος/η, ή συνήθως κάνει ότι του/της ζητούν οι ενήλικες - v_y', 'Τον/την απασχολεί το παραμικρό, συχνά φαίνεται ανήσυχος/η - v_y',\
                           'Είναι πρόθυμος/η να βοηθήσει κάποιον που είναι πληγωμένος, αναστατωμένος,στεναχωρημένος, άρρωστος - v_y', 'Προθυμοποιείται συχνά να βοηθήσει τους άλλους (γονείς, καθηγητές, άλλα παιδιά) - v_y',\
                           'Κλέβει από το σπίτι, το σχολείο ή αλλού - v_y', 'Πριν κάνει κάτι το μελετάει προσεκτικά - v_y', 'Έχει πολλούς φόβους, τρομάζει εύκολα - v_y',\
                           'Τα πηγαίνει καλύτερα με τους ενήλικες παρά με τα παιδιά - v_y', 'Φέρνει σε πέρας μία εργασία, έχει καλή προσοχή - v_y'])

In [32]:
#delete foodpractices columns(only need score) pre
data = data.drop(columns=['Περιορίζω την πρόσληψη τροφίμων από το παιδί μου, που μπορούν να το παχύνουν - v_x', 'Εάν το παιδί μου φάει περισσότερο από το συνηθισμένο σε ένα γεύμα, προσπαθώ να περιορίσω την πρόσληψη τροφής στο επόμενο γεύμα. - v_x',\
                          'Επιτρέπετε στο παιδί σας να τρώει σνακ όποτε  αυτό επιθυμεί; - v_x', 'Δεν δίνω γλυκά/επιδόρπια στο παιδί μου ως αντίδραση/ αποτέλεσμα της κακής του/της συμπεριφοράς. - v_x', 'Εάν στο παιδί σας δεν αρέσει το φαγητό που του σερβίρετε, ετοιμάζετε κάτι άλλο; - v_x',\
                          'Υπάρχουν συγκεκριμένα φαγητά που το παιδί μου δεν πρέπει να τρώει για μα μην παχύνει. - v_x', 'Ενθαρρύνετε το παιδί σας να τρώει υγιεινά φαγητά πριν την κατανάλωση ανθυγιεινών φαγητών; - v_x', 'Εάν το παιδί μου τρώει πολύ λίγο φαγητό, προσπαθώ να το κάνω να τρώει περισσότερο - v_x',\
                          'Επιτρέπετε στο παιδί σας να σηκωθεί από το τραπέζι όταν αυτό έχει χορτάσει, ακόμα και εάν τα άλλα μέλη της οικογένειας, δεν έχουν τελειώσει το γεύμα τους; - v_x', 'Ενθαρρύνω το παιδί μου να τρώει ποικιλία τροφίμων. - v_x', 'Τα περισσότερα φαγητά που υπάρχουν στο σπίτι είναι υγιεινά. - v_x',\
                          'Δεν επιτρέπω στο παιδί μου να τρώει ανάμεσα στα γεύματα, επειδή δεν θέλω να παχύνει. - v_x', 'Πρέπει να είμαι σίγουρος/η πως το παιδί μου δεν τρώει πάρα πολλά από τα αγαπημένα του τρόφιμα. - v_x', 'Επιτρέπω στο παιδί μου να βοηθάει στην προετοιμασία των οικογενειακών γευμάτων. - v_x',\
                          'Κάνω συχνά δίαιτα στο παιδί μου για να ελέγχω το βάρος του. - v_x', 'Προσφέρω στο παιδί μου το αγαπημένο του φαγητό ως ανταμοιβή για την καλή του συμπεριφορά. - v_x', 'Καταναλώνω υγιεινά τρόφιμα μπροστά στο παιδί, για να λειτουργήσω ως πρότυπο μίμησης γι’ αυτό. - v_x',\
                          'Υπάρχει ποικιλία υγιεινών φαγητών διαθέσιμα στο παιδί μου σε  κάθε γεύμα που σερβίρουμε στο σπίτι μας. - v_x', 'Εάν δεν καθοδηγούσα ή έλεγχα τη διατροφή του παιδιού μου, θα έτρωγε πάρα πολύ μεγάλη ποσότητα από τα αγαπημένα του/της τρόφιμα/φαγητά. - v_x',\
                          'Πόσο παρακολουθείτε τα γλυκά (καραμέλες, παγωτό, κέικ, πίτες, αρτοσκευάσματα) που καταναλώνει το παιδί σας; - v_x', 'Πόσο παρακολουθείτε τα σνακ (πατατάκια, Doritos, τυρογαριδάκια) που καταναλώνει το παιδί σας; - v_x', 'Πόσο παρακολουθείτε τα υψηλά σε λιπαρά τρόφιμα που καταναλώνει το παιδί σας; - v_x',\
                          'Πόσο παρακολουθείτε τα ποτά με ζάχαρη (πορτοκαλάδα, κόκα κόλα, λεμονάδα, γκαζόζα, χυμούς εμπορίου) που καταναλώνει το παιδί σας; - v_x', 'Επιτρέπετε στο παιδί σας να τρώει ό,τι θέλει; - v_x',\
                          'Επιτρέπετε στο παιδί σας να επιλέξει όποιο φαγητό θέλει, από αυτά που σερβίρετε στο δείπνο; - v_x', 'Όταν το παιδί σας γκρινιάζει, το πρώτο πράγμα που κάνετε είναι να του/της δίνετε κάτι να φάει ή να πιεί; - v_x',\
                          'Δίνετε στο παιδί σας κάτι να φάει ή να πιεί όταν βαριέται παρόλο που πιστεύετε πως δεν πεινάει; - v_x', 'Δίνετε στο παιδί σας κάτι να φάει ή να πιεί όταν  είναι αναστατωμένο, παρόλο που πιστεύετε πως δεν πεινάει; - v_x',\
                          'Προσπαθώ να καταναλώνω υγιεινά τρόφιμα μπροστά στο παιδί μου, ακόμα και εάν δεν είναι τα αγαπημένα μου. - v_x', 'Προσφέρω γλυκά (καραμέλες, παγωτό, κέικ, πίτες, αρτοσκευάσματα) στο παιδί μου σαν ανταμοιβή για την καλή του/της συμπεριφορά. - v_x',\
                          'Δείχνω στο παιδί μου πόσο πολύ απολαμβάνω να καταναλώνω υγιεινά τρόφιμα. - v_x', 'Προσπαθώ να δείχνω ενθουσιασμό όταν καταναλώνω υγιεινά τρόφιμα. - v_x', 'Συζητάω με το παιδί μου γιατί είναι σημαντικό να τρώει υγιεινά φαγητά. - v_x',\
                          'Ενθαρρύνω το παιδί μου να δοκιμάζει καινούργια φαγητά. - v_x', '. Όταν το παιδί μου λέει ότι έχει τελειώσει το φαγητό του, προσπαθώ να το κάνω να φάει ακόμα μία δύο μπουκιές. - v_x',\
                          'Ενθαρρύνω το παιδί μου να τρώει λιγότερο, για την αποφυγή αύξησης του σωματικού του/της βάρους. - v_x', 'Λέω στο παιδί μου πως το υγιεινό φαγητό έχει ωραία γεύση (είναι νόστιμο). - v_x',\
                          'Δίνω στο παιδί μου μικρές μερίδες στα γεύματα ώστε να ελέγχω το βάρος του. - v_x', 'Εάν δεν καθοδηγούσα/ έλεγχα τη διατροφή του παιδιού μου, θα έτρωγε πάρα πολλά ανθυγιεινά, πρόχειρα φαγητά (π.χ. fastfood, σουβλάκι, πίτσα). - v_x',\
                          'Συζητάω με το παιδί μου για τη διατροφική  αξία  των τροφίμων. - v_x', 'Εάν το παιδί μου πει «δεν πεινάω», προσπαθώ να τον/την κάνω να φάει ούτως ή άλλως. - v_x', 'Ενθαρρύνω το παιδί μου να συμμετέχει στην αγορά των  τροφίμων. - v_x'])

In [33]:
#delete mother columns(only need score) post
data =  data.drop(columns=['Μαλώνω και φωνάζω στο παιδί μου όταν φέρεται άσχημα. - v_x', 'Δείχνω στο παιδί μου κατανόηση όταν είναι αναστατωμένο. - v_x',\
                           'Υποκύπτω στις επιθυμίες του παιδιού μου όταν επιμένει έντονα σε αυτές. - v_x', 'Επαινώ το παιδί μου όταν είναι καλό και υπάκουο. - v_x',\
                           'Απειλώ περισσότερο το παιδί μου με τιμωρία παρά το τιμωρώ. - v_x', 'Ξεσπάω το θυμό μου στο παιδί μου. - v_x', 'Δέρνω το παιδί μου όταν δεν υπακούει. - v_x',\
                           'Λαμβάνω υπόψη μου τις επιθυμίες του παιδιού μου όταν κάνω οικογενειακά σχέδια. - v_x', 'Επιτρέπω στο παιδί μου να πει τη γνώμη του για τους κανόνες που υπάρχουν στην οικογένεια. - v_x',\
                           'Λέω στο παιδί μου με ποιο τρόπο θα το τιμωρήσω, αλλά δεν εφαρμόζω την τιμωρία. - v_x', 'Κατανοώ τα συναισθήματα του παιδιού μου. - v_x', 'Χρησιμοποιώ την τιμωρία ως μέσο πειθαρχίας. - v_x',\
                           'Λαμβάνω υπόψη μου τι θέλει το παιδί μου, πριν του ζητήσω να κάνει κάτι. - v_x', 'Λέω στο παιδί μου πως νοιώθω, όταν συμπεριφέρεται καλά ή όταν συμπεριφέρεται άσχημα. - v_x',\
                           'Παροτρύνω το παιδί μου να συζητά μαζί μου τα προβλήματά του. - v_x', 'Πιστεύω ότι είναι δύσκολο να μάθω στο παιδί μου να πειθαρχεί. - v_x', 'Ζητώ από το παιδί μου να εκφράζει τη γνώμη του ακόμα και όταν διαφωνεί μαζί μου. - v_x',\
                           'Τιμωρώ το παιδί μου απαγορεύοντάς του να κάνει κάτι που του αρέσει χωρίς να του δίνω εξηγήσεις. - v_x', 'Λέω στο παιδί μου πόσο σημαντικό είναι να ακολουθεί τους κανόνες της οικογένειας. - v_x',\
                           'Κάνω συστάσεις στο παιδί μου για να βελτιωθεί η συμπεριφορά του - v_x', 'Εξηγώ στο παιδί μου γιατί πρέπει να τηρούνται οι κανόνες της οικογένειας - v_x', 'Κακομαθαίνω το παιδί μου - v_x',\
                           'Δείχνω τρυφερότητα στο παιδί μου - v_x', 'Χρησιμοποιώ απειλές για να τιμωρήσω το παιδί μου χωρίς να του δίνω εξηγήσεις - v_x', 'Βοηθώ το παιδί μου να καταλάβει τις συνέπειες μιας κακής πράξης του μέσα από διάλογο και συζήτηση - v_x',\
                           'Τιμωρώ το παιδί μου στέλνοντάς το στο δωμάτιό του χωρίς να του δίνω εξηγήσεις - v_x', 'Συζητώ με το παιδί μου για τις συνέπειες των πράξεών του - v_x', 'Ασκώ κριτική στο παιδί μου όταν η συμπεριφορά του δεν ανταποκρίνεται στις προσδοκίες μου - v_x',\
                           'Χαστουκίζω το παιδί μου όταν δεν συμπεριφέρεται σωστά - v_x'])

In [34]:
#delete strengthdiff columns(only need score) post
data =  data.drop(columns=['Έχει τουλάχιστον ένα φίλο - v_x', 'Συνεχώς στριφογυρίζει νευρικά ή δεν στέκεται ήσυχος/η, έχει νευρικότητα - v_x',
                           'Συχνά είναι δυστυχισμένος/η, αποκαρδιωμένος/η ή κλαίει - v_x', 'Συχνά μαλώνει με τα άλλα παιδιά ή τα κοροϊδεύει, τα φοβερίζει, τα κτυπά - v_x',\
                           'Η προσοχή του/της διασπάται εύκολα, δυσκολεύεται να συγκεντρωθεί - v_x', 'Γενικά είναι συμπαθής στα άλλα παιδιά - v_x',\
                           'Είναι καλός/ή με τα μικρότερα παιδιά - v_x', 'Σε καινούριες καταστάσεις είναι νευρικός/η ή δείχνει συμπεριφορά προσκόλλησης, εύκολα χάνει την αυτοπεποίθησή του/της - v_x',\
                           'Τα άλλα παιδιά τον/την έχουν στο μάτι ή τον/την κοροϊδεύουν, τον/την φοβερίζουν, τον/την χτυπούν - v_x', 'Συχνά λέει ψέμματα ή εξαπατά - v_x',\
                           'Λαμβάνει υπόψη τα συναισθήματα των άλλων - v_x', 'Ανήσυχος/η και υπερδραστήριος/α, δεν μπορεί να παραμείνει ήρεμος/η, ακίνητος/η για πολύ ώρα - v_x',\
                           'Παραπονείται συχνά για πονοκεφάλους, πόνους στο στομάχι ή αδιαθεσία - v_x', 'Μοιράζεται εύκολα με τα άλλα παιδιά (κεράσματα, παιχνίδια, μολύβια κτλ) - v_x',\
                           'Συχνά έχει ξεσπάσματα νεύρων ή είναι ευέξαπτος - v_x', 'Μάλλον μοναχικός/η, τείνει να παίζει μόνος/η - v_x',\
                           'Γενικά είναι υπάκουος/η, ή συνήθως κάνει ότι του/της ζητούν οι ενήλικες - v_x', 'Τον/την απασχολεί το παραμικρό, συχνά φαίνεται ανήσυχος/η - v_x',\
                           'Είναι πρόθυμος/η να βοηθήσει κάποιον που είναι πληγωμένος, αναστατωμένος,στεναχωρημένος, άρρωστος - v_x', 'Προθυμοποιείται συχνά να βοηθήσει τους άλλους (γονείς, καθηγητές, άλλα παιδιά) - v_x',\
                           'Κλέβει από το σπίτι, το σχολείο ή αλλού - v_x', 'Πριν κάνει κάτι το μελετάει προσεκτικά - v_x', 'Έχει πολλούς φόβους, τρομάζει εύκολα - v_x',\
                           'Τα πηγαίνει καλύτερα με τους ενήλικες παρά με τα παιδιά - v_x', 'Φέρνει σε πέρας μία εργασία, έχει καλή προσοχή - v_x'])

In [35]:
#delete foodpractices columns(only need score) post
data = data.drop(columns=['Περιορίζω την πρόσληψη τροφίμων από το παιδί μου, που μπορούν να το παχύνουν - v_y', 'Εάν το παιδί μου φάει περισσότερο από το συνηθισμένο σε ένα γεύμα, προσπαθώ να περιορίσω την πρόσληψη τροφής στο επόμενο γεύμα. - v_y',\
                          'Επιτρέπετε στο παιδί σας να τρώει σνακ όποτε  αυτό επιθυμεί; - v_y', 'Δεν δίνω γλυκά/επιδόρπια στο παιδί μου ως αντίδραση/ αποτέλεσμα της κακής του/της συμπεριφοράς. - v_y', 'Εάν στο παιδί σας δεν αρέσει το φαγητό που του σερβίρετε, ετοιμάζετε κάτι άλλο; - v_y',\
                          'Υπάρχουν συγκεκριμένα φαγητά που το παιδί μου δεν πρέπει να τρώει για μα μην παχύνει. - v_y', 'Ενθαρρύνετε το παιδί σας να τρώει υγιεινά φαγητά πριν την κατανάλωση ανθυγιεινών φαγητών; - v_y', 'Εάν το παιδί μου τρώει πολύ λίγο φαγητό, προσπαθώ να το κάνω να τρώει περισσότερο - v_y',\
                          'Επιτρέπετε στο παιδί σας να σηκωθεί από το τραπέζι όταν αυτό έχει χορτάσει, ακόμα και εάν τα άλλα μέλη της οικογένειας, δεν έχουν τελειώσει το γεύμα τους; - v_y', 'Ενθαρρύνω το παιδί μου να τρώει ποικιλία τροφίμων. - v_y', 'Τα περισσότερα φαγητά που υπάρχουν στο σπίτι είναι υγιεινά. - v_y',\
                          'Δεν επιτρέπω στο παιδί μου να τρώει ανάμεσα στα γεύματα, επειδή δεν θέλω να παχύνει. - v_y', 'Πρέπει να είμαι σίγουρος/η πως το παιδί μου δεν τρώει πάρα πολλά από τα αγαπημένα του τρόφιμα. - v_y', 'Επιτρέπω στο παιδί μου να βοηθάει στην προετοιμασία των οικογενειακών γευμάτων. - v_y',\
                          'Κάνω συχνά δίαιτα στο παιδί μου για να ελέγχω το βάρος του. - v_y', 'Προσφέρω στο παιδί μου το αγαπημένο του φαγητό ως ανταμοιβή για την καλή του συμπεριφορά. - v_y', 'Καταναλώνω υγιεινά τρόφιμα μπροστά στο παιδί, για να λειτουργήσω ως πρότυπο μίμησης γι’ αυτό. - v_y',\
                          'Υπάρχει ποικιλία υγιεινών φαγητών διαθέσιμα στο παιδί μου σε  κάθε γεύμα που σερβίρουμε στο σπίτι μας. - v_y', 'Εάν δεν καθοδηγούσα ή έλεγχα τη διατροφή του παιδιού μου, θα έτρωγε πάρα πολύ μεγάλη ποσότητα από τα αγαπημένα του/της τρόφιμα/φαγητά. - v_y',\
                          'Πόσο παρακολουθείτε τα γλυκά (καραμέλες, παγωτό, κέικ, πίτες, αρτοσκευάσματα) που καταναλώνει το παιδί σας; - v_y', 'Πόσο παρακολουθείτε τα σνακ (πατατάκια, Doritos, τυρογαριδάκια) που καταναλώνει το παιδί σας; - v_y', 'Πόσο παρακολουθείτε τα υψηλά σε λιπαρά τρόφιμα που καταναλώνει το παιδί σας; - v_y',\
                          'Πόσο παρακολουθείτε τα ποτά με ζάχαρη (πορτοκαλάδα, κόκα κόλα, λεμονάδα, γκαζόζα, χυμούς εμπορίου) που καταναλώνει το παιδί σας; - v_y', 'Επιτρέπετε στο παιδί σας να τρώει ό,τι θέλει; - v_y',\
                          'Επιτρέπετε στο παιδί σας να επιλέξει όποιο φαγητό θέλει, από αυτά που σερβίρετε στο δείπνο; - v_y', 'Όταν το παιδί σας γκρινιάζει, το πρώτο πράγμα που κάνετε είναι να του/της δίνετε κάτι να φάει ή να πιεί; - v_y',\
                          'Δίνετε στο παιδί σας κάτι να φάει ή να πιεί όταν βαριέται παρόλο που πιστεύετε πως δεν πεινάει; - v_y', 'Δίνετε στο παιδί σας κάτι να φάει ή να πιεί όταν  είναι αναστατωμένο, παρόλο που πιστεύετε πως δεν πεινάει; - v_y',\
                          'Προσπαθώ να καταναλώνω υγιεινά τρόφιμα μπροστά στο παιδί μου, ακόμα και εάν δεν είναι τα αγαπημένα μου. - v_y', 'Προσφέρω γλυκά (καραμέλες, παγωτό, κέικ, πίτες, αρτοσκευάσματα) στο παιδί μου σαν ανταμοιβή για την καλή του/της συμπεριφορά. - v_y',\
                          'Δείχνω στο παιδί μου πόσο πολύ απολαμβάνω να καταναλώνω υγιεινά τρόφιμα. - v_y', 'Προσπαθώ να δείχνω ενθουσιασμό όταν καταναλώνω υγιεινά τρόφιμα. - v_y', 'Συζητάω με το παιδί μου γιατί είναι σημαντικό να τρώει υγιεινά φαγητά. - v_y',\
                          'Ενθαρρύνω το παιδί μου να δοκιμάζει καινούργια φαγητά. - v_y', '. Όταν το παιδί μου λέει ότι έχει τελειώσει το φαγητό του, προσπαθώ να το κάνω να φάει ακόμα μία δύο μπουκιές. - v_y',\
                          'Ενθαρρύνω το παιδί μου να τρώει λιγότερο, για την αποφυγή αύξησης του σωματικού του/της βάρους. - v_y', 'Λέω στο παιδί μου πως το υγιεινό φαγητό έχει ωραία γεύση (είναι νόστιμο). - v_y',\
                          'Δίνω στο παιδί μου μικρές μερίδες στα γεύματα ώστε να ελέγχω το βάρος του. - v_y', 'Εάν δεν καθοδηγούσα/ έλεγχα τη διατροφή του παιδιού μου, θα έτρωγε πάρα πολλά ανθυγιεινά, πρόχειρα φαγητά (π.χ. fastfood, σουβλάκι, πίτσα). - v_y',\
                          'Συζητάω με το παιδί μου για τη διατροφική  αξία  των τροφίμων. - v_y', 'Εάν το παιδί μου πει «δεν πεινάω», προσπαθώ να τον/την κάνω να φάει ούτως ή άλλως. - v_y', 'Ενθαρρύνω το παιδί μου να συμμετέχει στην αγορά των  τροφίμων. - v_y'])

In [36]:
data = data.drop(columns=['Σκέφτομαι ότι καίω θερμίδες όταν γυμνάζομαι - v', 'Είμαι απασχολημένος/η με την επιθυμία να είμαι πιο αδύνατος/η - v',\
                          'Απασχολούμαι με την σκέψη ότι έχω λίπος στο σώμα μου - v', 'Άλλοι άνθρωποι πιστεύουν ότι είμαι πολύ αδύνατος/η - v',\
                          'Αποφεύγω τροφές με ζάχαρη - v', 'Όταν τρώω τα γεύματά μου αργώ περισσότερο από τους άλλους - v',\
                          'Νοιώθω ότι το φαγητό ελέγχει την ζωή μου - v', 'Τρώω διαιτητικές τροφές - v', 'Νοιώθω ότι οι άλλοι με πιέζουν να φάω - v',\
                          'Έχω αυτο-έλεγχο σχετικά με το φαγητό - v', 'Τρομάζω στην ιδέα να είμαι υπέρβαρος/η - v', 'Αποφεύγω να τρώω όταν πεινάω - v',\
                          'Πιάνω τον εαυτό μου απασχολημένο με το φαγητό - v', 'Κόβω την τροφή μου σε μικρά κομμάτια - v',\
                          'Γνωρίζω το θερμιδικό περιεχόμενο των τροφών που καταναλώνω - v', 'Συγκεκριμένα αποφεύγω τροφές που περιέχουν πολλούς υδατάνθρακες (ψωμί, ρύζι, πατάτες, κλπ.) - v',\
                          'Νοιώθω ότι οι άλλοι θα προτιμούσαν να έτρωγα περισσότερο - v', 'Προκαλώ εμετό μετά το φαγητό - v', 'Νοιώθω πολύ ένοχος/η μετά το φαγητό - v',\
                          'Ξοδεύω πολύ χρόνο και σκέψη στο φαγητό - v', 'Κάνω δίαιτα - v', 'Νοιώθω άβολα όταν τρώω γλυκά - v', 'Νοιώθω την ανάγκη να κάνω εμετό μετά το φαγητό - v',\
                          'Μου αρέσει να είναι άδειο το στομάχι μου - v', 'Έχετε επεισόδια υπερφαγίας όπου νιώθετε ότι δεν μπορείτε να σταματήσετε; - v',\
                          'Απολαμβάνω να δοκιμάζω νέα λιπαρά /παχυντικά φαγητά - v'])

In [37]:
# Print part of final data
print(data.shape)
print(data.loc[data['pid'] == "61d6ce215828a5043783a1f5"])

(2773, 137)
                           pid   timestamp  sleepAsleepMinutes  distance  \
2183  61d6ce215828a5043783a1f5  2022-01-04                   0         0   
2184  61d6ce215828a5043783a1f5  2022-01-05                   0         0   
2185  61d6ce215828a5043783a1f5  2022-01-06                   0     21760   
2186  61d6ce215828a5043783a1f5  2022-01-07                 560    645790   
2187  61d6ce215828a5043783a1f5  2022-01-08                 463    798030   
...                        ...         ...                 ...       ...   
2348  61d6ce215828a5043783a1f5  2022-04-15                   0         0   
2349  61d6ce215828a5043783a1f5  2022-04-15                   0         0   
2350  61d6ce215828a5043783a1f5  2022-04-15                   0         0   
2351  61d6ce215828a5043783a1f5  2022-04-15                   0         0   
2352  61d6ce215828a5043783a1f5  2022-04-15                   0         0   

      steps  target weight  home  junk  ΙΚΤΕΡΟΣ - v  ...  \
2183      0    

# **Pre-processing**

In [38]:
#obj = data.select_dtypes(include=[np.object])
for i in data.columns: 
  if "Score" in i:
    print(i)

Score_sd_post - v
Score - v
Score_fp - v
Score_mother_post - v
Score_fp_post - v
Score_mother - v
Score_psycho - v
Score_sd - v


In [39]:
for name,col_type in zip(data.columns,data.dtypes):
    print (name,col_type)

pid object
timestamp object
sleepAsleepMinutes int64
distance int64
steps int64
target float64
weight object
home float64
junk float64
ΙΚΤΕΡΟΣ - v float64
ΥΠΟΓΛΥΚΑΙΜΙΑ - v float64
ΠΑΡΑΜΟΝΗ ΣΕ ΜΕΝ / ΜΑΦ - v float64
HbA1c - v object
Chol - v float64
HDL - v float64
LDL - v float64
TG - v float64
Lp(a) - v object
SGOT - v float64
SGPT - v float64
γGT - v object
FSH - v object
TSH - v object
UA - v object
25 (OH) vitamin D - v object
FreeT4 - v object
Κορτιζόλη - v object
Ινσουλίνη - v object
Τεστοστερόνη - v object
ACTH - v object
LH - v object
Οιστραδιόλη - v object
Γλυκόζη - v float64
ΗΛΙΚΙΑ ΜΗΤΡΟΣ - v float64
ΒΑΡΟΣ ΜΗΤΡΟΣ - v float64
ΥΨΟΣ ΜΗΤΡΟΣ - v float64
ΝΟΣΗΜΑΤΑ ΜΗΤΡΟΣ - v object
ΗΛΙΚΙΑ ΠΑΤΡΟΣ - v float64
ΒΑΡΟΣ ΠΑΤΡΟΣ - v float64
ΥΨΟΣ ΠΑΤΡΟΣ - v float64
ΝΟΣΗΜΑΤΑ ΠΑΤΡΟΣ - v object
ΑΔΕΛΦΙΑ - v float64
BMI - v_x object
BMI - v_y float64
Score_sd_post - v object
Score - v float64
Score_fp - v object
ΜΕΣΟΔΙΑΣΤΗΜΑΤΑ ΜΕΤΑΞΥ ΓΕΥΜΑΤΩΝ: - v float64
ΑΡΙΘΜΟΣ ΜΙΚΡΟΓΕΥΜΑΤΩΝ / ΣΝΑΚ - v float64
Πό

### Remove duplicates

In [40]:
print(data.shape)
data = data.drop_duplicates()
print(data.shape)

(2773, 137)
(2649, 137)


## Convert column types

### Convert to Float

In [41]:
#split bmi mother/father from BMI - v_y
data['BMI - v_y'] = data['BMI - v_y'].astype('str')
data[['BMI_Mother', 'BMI_Father']] = data['BMI - v_y'].str.split('.', expand=True)
data['BMI_Mother'] = data['BMI_Mother'].astype('str')
data['BMI_Father'] = data['BMI_Father'].astype('str')


In [42]:
del data['BMI - v_x']
del data['BMI - v_y']

In [43]:
#fix score columns

scores = ['Score_mother - v', 'Score_fp - v', 'Score_fp_post - v', 'Score_mother_post - v', 'Score_sd_post - v', 'Score_sd - v']

for i in scores:
  data[i] = data[i].astype(str)

data[['Mother_Yp', 'Mother_Aus', 'Mother_Ep', 'Mother_Aut']] = data['Score_mother - v'].str.split(',', expand=True)
data[['sd_Tot', 'sd_Emo', 'sd_Cond', 'sd_Hyper', 'sd_Peer', 'sd_Beh']] = data['Score_sd - v'].str.split(',', expand=True)
data[['fp_Mon', 'fp_Con', 'fp_Reg', 'fp_Guid', 'fp_Res', 'fp_Press']] = data['Score_fp - v'].str.split(',', expand=True)
data[['Post_Mother_Yp', 'Post_Mother_Aus', 'Post_Mother_Ep', 'Post_Mother_Aut']] = data['Score_mother_post - v'].str.split(',', expand=True)
data[['Post_sd_Tot', 'Post_sd_Emo', 'Post_sd_Cond', 'Post_sd_Hyper', 'Post_sd_Peer', 'Post_sd_Beh']] = data['Score_sd_post - v'].str.split(',', expand=True)
data[['Post_fp_Mon', 'Post_fp_Con', 'Post_fp_Reg', 'Post_fp_Guid', 'Post_fp_Res', 'Post_fp_Press']] = data['Score_fp_post - v'].str.split(',', expand=True)

In [44]:
sd = ['sd_Tot', 'sd_Emo', 'sd_Cond', 'sd_Hyper', 'sd_Peer', 'sd_Beh', 'Post_sd_Tot', 'Post_sd_Emo', 'Post_sd_Cond', 'Post_sd_Hyper', 'Post_sd_Peer', 'Post_sd_Beh']

data[['sd_Tot', '1']] = data['sd_Tot'].str.split(' ', expand=True)
data[['sd_Emo', '2']] = data['sd_Emo'].str.split(' ', expand=True)
data[['sd_Cond', '3']] = data['sd_Cond'].str.split(' ', expand=True)
data[['sd_Hyper', '4']] = data['sd_Hyper'].str.split(' ', expand=True)
data[['sd_Peer', '5']] = data['sd_Peer'].str.split(' ', expand=True)
data[['sd_Beh', '6']] = data['sd_Beh'].str.split(' ', expand=True)
data[['Post_sd_Tot', '7']] = data['Post_sd_Tot'].str.split(' ', expand=True)
data[['Post_sd_Emo', '8']] = data['Post_sd_Emo'].str.split(' ', expand=True)
data[['Post_sd_Cond', '9']] = data['Post_sd_Cond'].str.split(' ', expand=True)
data[['Post_sd_Hyper', '10']] = data['Post_sd_Hyper'].str.split(' ', expand=True)
data[['Post_sd_Peer', '11']] = data['Post_sd_Peer'].str.split(' ', expand=True)
data[['Post_sd_Beh', '12']] = data['Post_sd_Beh'].str.split(' ', expand=True)

data = data.drop(columns=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])
data = data.drop(columns=['Score_mother - v', 'Score_fp - v', 'Score_fp_post - v', 'Score_mother_post - v', 'Score_sd_post - v', 'Score_sd - v'])

In [45]:
#convert to float

data_float = ['weight', 'βάρος γέννησης - v', 'μήκος σώματος - v', 'περίμετρος κεφαλής - v', 'ΒΑΡΟΣ ΜΗΤΕΡΑΣ ΠΡΟ ΚΥΗΣΗΣ - v', 'ΒΑΡΟΣ ΜΗΤΡΟΣ - v', 'ΥΨΟΣ ΜΗΤΡΟΣ - v',\
              'ΒΑΡΟΣ ΠΑΤΡΟΣ - v', 'ΥΨΟΣ ΠΑΤΡΟΣ - v', 'BMI_Mother', 'BMI_Father', 'ΠΡΟΣΛΗΨΗ ΒΑΡΟΥΣ ΚΑΤΑ ΤΗΝ ΕΓΚΥΜΟΣΥΝΗ - v', 'HbA1c - v', 'Chol - v', 'HDL - v', 'LDL - v',\
              'TG - v', 'Lp(a) - v', 'SGOT - v', 'SGPT - v', 'γGT - v', 'FSH - v', 'TSH - v', 'UA - v', '25 (OH) vitamin D - v', 'FreeT4 - v', 'Κορτιζόλη - v',\
              'Ινσουλίνη - v', 'Τεστοστερόνη - v', 'ACTH - v', 'LH - v', 'Οιστραδιόλη - v', 'Γλυκόζη - v', 'Ύψος σε μέτρα - v', 'Βάρος σε κιλά - v',\
              'Score - v', 'Score_psycho - v', 'BMI_Mother', 'BMI_Father', 'Mother_Yp', 'Mother_Aus', 'Mother_Ep', 'Mother_Aut',\
              'sd_Tot', 'sd_Emo', 'sd_Cond', 'sd_Hyper', 'sd_Peer', 'sd_Beh', 'fp_Mon', 'fp_Con', 'fp_Reg', 'fp_Guid', 'fp_Res', 'fp_Press',\
              'Post_Mother_Yp', 'Post_Mother_Aus', 'Post_Mother_Ep', 'Post_Mother_Aut', 'Post_sd_Tot', 'Post_sd_Emo', 'Post_sd_Cond', 'Post_sd_Hyper', 'Post_sd_Peer', 'Post_sd_Beh',\
              'Post_fp_Mon', 'Post_fp_Con', 'Post_fp_Reg', 'Post_fp_Guid', 'Post_fp_Res', 'Post_fp_Press']

In [46]:
for i in data_float:
  data[i] = data[i].astype(str).str.replace(',','.')
  data[i] = data[i].astype(str).str.replace('<','')
  data[i] = data[i].astype(str).str.replace('Ο','0')
  data[i] = data[i].astype(str).str.replace(' ','')
  data[i] = data[i].astype(str).str.replace('-','nan')
  data[i] = data[i].astype(str).str.replace('None','nan')
  data[i] = data[i].replace('nan', np.nan) 

In [47]:
for col in data_float:
  data[col]= data[col].astype(float)             

  ### Convert to String and Date

In [48]:
#convert to string
data["pid"]= data["pid"].astype(str)

### Convert to Int

In [49]:
data['ΠΡΟΣΩΠΑ ΦΡΟΝΤΙΔΑΣ  ΣΙΤΙΣΗΣ ΠΑΙΔΙΟΥ: - v'].unique()

array([nan, '0,1', '0,2', '0,3', '0', '0,1,2'], dtype=object)

In [50]:
del data['ΠΡΟΣΩΠΑ ΦΡΟΝΤΙΔΑΣ  ΣΙΤΙΣΗΣ ΠΑΙΔΙΟΥ: - v']

In [51]:
#convert to int

data_int = ['target','home','junk','ΔΙΑΡΚΕΙΑ  θΗΛΑΣΜΟΥ ΣΕ ΜΗΝΕΣ - v', 'ΔΙΑΡΚΕΙΑ ΑΠΟΚΛΕΙΣΤΙΚΟΥ ΘΗΛΑΣΜΟΥ (ΜΗΝΕΣ): - v', 'ΔΙΑΡΚΕΙΑ ΜΙΚΤΟΥ ΘΗΛΑΣΜΟΥ (ΜΗΝΕΣ): - v', 'Φύλο - v', 'Ηλικία - v',\
            'Με βάση το ύψος σας και το σωματικό σας βάρος, θεωρείτε πως είστε - v', 'Ποια είναι η οικογενειακή σας κατάσταση; - v', 'Ποια είναι η εθνικότητά σας*; - v',\
            'Ποιο είναι το μορφωτικό σας επίπεδο; - v', 'Ποια είναι η εργασία σας; (σε περίπτωση συνταξιοδότησης σημειώνετε την απασχόληση πριν από τη συνταξιοδότηση) - v',\
            'Είστε ικανοποιημένος με το εισόδημά σας; - v', 'Ποιο είναι το ακαθάριστο ετήσιο εισόδημά σας; (η ερώτηση αυτή είναι προαιρετική) - v', 'Είστε καπνιστής; - v',\
            'Το οικογενειακό σας εισόδημα καλύπτει τις ανάγκες σας; - v', 'Ασκείστε; Πόσο συχνά; - v', 'Καταναλώνετε αλκοολούχα ποτά; - v',\
            'Νοσείτε από κάποια χρόνια ασθένεια (πχ. Υπέρταση, Δυσλιπιδαιμία, Διαβήτη,); - v', 'Πόσο καλή είναι η υγεία σας; - v']       

In [52]:
for i in data_int:
  data[i] = data[i].astype(str).str.replace(',','.')
  data[i] = data[i].astype(str).str.replace('<','')
  data[i] = data[i].astype(str).str.replace('Ο','0')
  data[i] = data[i].astype(str).str.replace(' ','')
  data[i] = data[i].astype(str).str.replace('-','nan')
  data[i] = data[i].replace('nan',np.NaN)

In [53]:
for col in data_int:
  data[col] = pd.to_numeric(data[col])
  data[col]= data[col].astype(int, errors='ignore')            

In [54]:
data.dtypes

pid                    object
timestamp              object
sleepAsleepMinutes      int64
distance                int64
steps                   int64
                       ...   
Post_fp_Con           float64
Post_fp_Reg           float64
Post_fp_Guid          float64
Post_fp_Res           float64
Post_fp_Press         float64
Length: 162, dtype: object

## Fix missing 


### Numerical

In [55]:
#columns that can be replaced by mean values
data_mean = ['weight','sleepAsleepMinutes', 'Εβδομάδες κύησης - v', 'βάρος γέννησης - v', 'μήκος σώματος - v', 'περίμετρος κεφαλής - v', 'ΒΑΡΟΣ ΜΗΤΕΡΑΣ ΠΡΟ ΚΥΗΣΗΣ - v',\
             'ΠΡΟΣΛΗΨΗ ΒΑΡΟΥΣ ΚΑΤΑ ΤΗΝ ΕΓΚΥΜΟΣΥΝΗ - v', 'ΔΙΑΡΚΕΙΑ  θΗΛΑΣΜΟΥ ΣΕ ΜΗΝΕΣ - v', 'ΔΙΑΡΚΕΙΑ ΑΠΟΚΛΕΙΣΤΙΚΟΥ ΘΗΛΑΣΜΟΥ (ΜΗΝΕΣ): - v', 'ΔΙΑΡΚΕΙΑ ΜΙΚΤΟΥ ΘΗΛΑΣΜΟΥ (ΜΗΝΕΣ): - v',\
             'ΗΛΙΚΙΑ ΜΗΤΡΟΣ - v', 'ΒΑΡΟΣ ΜΗΤΡΟΣ - v', 'ΥΨΟΣ ΜΗΤΡΟΣ - v', 'ΗΛΙΚΙΑ ΠΑΤΡΟΣ - v', 'ΒΑΡΟΣ ΠΑΤΡΟΣ - v', 'ΥΨΟΣ ΠΑΤΡΟΣ - v', 'ΜΕΣΟΔΙΑΣΤΗΜΑΤΑ ΜΕΤΑΞΥ ΓΕΥΜΑΤΩΝ: - v',\
             'ΑΡΙΘΜΟΣ ΜΙΚΡΟΓΕΥΜΑΤΩΝ / ΣΝΑΚ - v', 'ΟΙΚΟΓΕΝΕΙΑΚΑ ΓΕΥΜΑΤΑ - v', 'ΑΡΙΘΜΟΣ ΚΥΡΙΩΝ ΓΕΥΜΑΤΩΝ - v', 'Ηλικία - v', 'HbA1c - v', 'Chol - v', 'HDL - v', 'LDL - v',\
             'TG - v', 'Lp(a) - v', 'SGOT - v', 'SGPT - v', 'γGT - v', 'FSH - v', 'TSH - v', 'UA - v', '25 (OH) vitamin D - v', 'FreeT4 - v', 'Κορτιζόλη - v', 'Ινσουλίνη - v',\
             'Τεστοστερόνη - v', 'ACTH - v', 'LH - v', 'Οιστραδιόλη - v', 'Γλυκόζη - v', 'Ύψος σε μέτρα - v', 'Βάρος σε κιλά - v',\
             'Score - v', 'Score_psycho - v', 'BMI_Mother', 'BMI_Father', 'Mother_Yp', 'Mother_Aus', 'Mother_Ep', 'Mother_Aut',\
             'sd_Tot', 'sd_Emo', 'sd_Cond', 'sd_Hyper', 'sd_Peer', 'sd_Beh', 'fp_Mon', 'fp_Con', 'fp_Reg', 'fp_Guid', 'fp_Res', 'fp_Press',\
             'Post_Mother_Yp', 'Post_Mother_Aus', 'Post_Mother_Ep', 'Post_Mother_Aut', 'Post_sd_Tot', 'Post_sd_Emo', 'Post_sd_Cond', 'Post_sd_Hyper', 'Post_sd_Peer', 'Post_sd_Beh',\
             'Post_fp_Mon', 'Post_fp_Con', 'Post_fp_Reg', 'Post_fp_Guid', 'Post_fp_Res', 'Post_fp_Press']

In [56]:
#Print columns and their missing values
for column in data.columns:
    print(column,": ",data[column].isnull().sum())  
        
data.replace('nan',np.NaN,inplace=True)

for col in data_mean:
  data[col].fillna(data[col].mean(), inplace=True)

print()
print()
#Print columns and their missing values after removing them
for column in data.columns:
    print(column,": ",data[column].isnull().sum())  

pid :  0
timestamp :  0
sleepAsleepMinutes :  0
distance :  0
steps :  0
target :  2453
weight :  2596
home :  2535
junk :  2581
ΙΚΤΕΡΟΣ - v :  1369
ΥΠΟΓΛΥΚΑΙΜΙΑ - v :  1369
ΠΑΡΑΜΟΝΗ ΣΕ ΜΕΝ / ΜΑΦ - v :  1369
HbA1c - v :  1469
Chol - v :  1369
HDL - v :  1369
LDL - v :  1369
TG - v :  1369
Lp(a) - v :  1369
SGOT - v :  1369
SGPT - v :  1369
γGT - v :  1369
FSH - v :  1457
TSH - v :  1465
UA - v :  1513
25 (OH) vitamin D - v :  1369
FreeT4 - v :  1465
Κορτιζόλη - v :  1369
Ινσουλίνη - v :  1425
Τεστοστερόνη - v :  1369
ACTH - v :  1457
LH - v :  1457
Οιστραδιόλη - v :  1457
Γλυκόζη - v :  1369
ΗΛΙΚΙΑ ΜΗΤΡΟΣ - v :  1369
ΒΑΡΟΣ ΜΗΤΡΟΣ - v :  1369
ΥΨΟΣ ΜΗΤΡΟΣ - v :  1369
ΝΟΣΗΜΑΤΑ ΜΗΤΡΟΣ - v :  1369
ΗΛΙΚΙΑ ΠΑΤΡΟΣ - v :  1369
ΒΑΡΟΣ ΠΑΤΡΟΣ - v :  1369
ΥΨΟΣ ΠΑΤΡΟΣ - v :  1369
ΝΟΣΗΜΑΤΑ ΠΑΤΡΟΣ - v :  1369
ΑΔΕΛΦΙΑ - v :  1369
Score - v :  1437
ΜΕΣΟΔΙΑΣΤΗΜΑΤΑ ΜΕΤΑΞΥ ΓΕΥΜΑΤΩΝ: - v :  1369
ΑΡΙΘΜΟΣ ΜΙΚΡΟΓΕΥΜΑΤΩΝ / ΣΝΑΚ - v :  1369
Πόσες ώρες περίπου παίζει βιντεοπαιχνίδια  το παιδί σας τις καθημερινές;

### Categorical



In [57]:
# Map categorical values
probmap = {'Υποθυρεοειδισμος κύησης': 1, 'Θρομβοφιλία': 1, 'Μερική αποκόλληση πλακούντα': 1, '-': 0}
data['ΑΛΛΑ ΠΡΟΒΛΗΜΑΤΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v'] = data['ΑΛΛΑ ΠΡΟΒΛΗΜΑΤΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v'].map(probmap)

nmmap = {'Προδιαβήτης': 1, 'Ιστορικό Ca μαστού': 1, '4,5,6': 1, '6': 1, 'Διπολική διαταραχή': 1, '0': 0}
data['ΝΟΣΗΜΑΤΑ ΜΗΤΡΟΣ - v'] = data['ΝΟΣΗΜΑΤΑ ΜΗΤΡΟΣ - v'].map(nmmap)

npmap = {'1,2': 1, '2': 1, '5': 1, '2,5': 1, 'Νευροεκφυλιστικό νόσημα': 1, '0': 0}
data['ΝΟΣΗΜΑΤΑ ΠΑΤΡΟΣ - v'] = data['ΝΟΣΗΜΑΤΑ ΠΑΤΡΟΣ - v'].map(npmap)

remap = {'0': 1, '1': 0}
data['ΡΥΘΜΟΣ ΣΙΤΙΣΗΣ - v'] = data['ΡΥΘΜΟΣ ΣΙΤΙΣΗΣ - v'].map(remap)

In [58]:
data_categ = ['ΚΥΗΣΗ - v', 'ΚΑΠΝΙΣΜΑ ΠΡΟ ΚΥΗΣΗΣ - v', 'Σ.Δ. ΚΥΗΣΗΣ - v', 'IUGR (ενδομήτρια καθυστέρηση αύξησης) - v', 'ΚΑΠΝΙΣΜΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v', 'ΑΛΛΑ ΠΡΟΒΛΗΜΑΤΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v',\
              'SGA - v', 'ΤΟΚΕΤΟΣ - v', 'ΙΚΤΕΡΟΣ - v', 'ΥΠΟΓΛΥΚΑΙΜΙΑ - v', 'ΠΑΡΑΜΟΝΗ ΣΕ ΜΕΝ / ΜΑΦ - v','ΘΗΛΑΣΜΟΣ - v', 'ΑΛΛΕΡΓΙΕΣ - v',\
              'ΦΑΡΜΑΚΑ / ΒΙΤΑΜΙΝΕΣ / ΣΥΜΠΛΗΡΩΜΑΤΑ - v', 'Χρόνια νοσήματα - v', 'Συγγενή νοσήματα - v', 'ΝΟΣΗΛΕΙΕΣ - v', 'ΝΟΣΗΜΑΤΑ ΜΗΤΡΟΣ - v', 'ΝΟΣΗΜΑΤΑ ΠΑΤΡΟΣ - v',\
              'ΑΔΕΛΦΙΑ - v', 'ΜΕΣΟΔΙΑΣΤΗΜΑΤΑ ΜΕΤΑΞΥ ΓΕΥΜΑΤΩΝ: - v', 'ΑΡΙΘΜΟΣ ΜΙΚΡΟΓΕΥΜΑΤΩΝ / ΣΝΑΚ - v',\
              'Πόσες ώρες περίπου παίζει βιντεοπαιχνίδια  το παιδί σας τις καθημερινές; - v', 'Πόσες ώρες περίπου την ημέρα (Σαββατοκύριακο ή αργίες)  πέρασε το παιδί σας μπροστά σε οθόνες; (π.χ. τηλεόραση, τάμπλετ, κινητό) - v',\
              'Πόσο συχνά τρώει το παιδί σας ψάρια ή θαλασσινά; - v', 'Πόσες ώρες την ημέρα περίπου κοιμάται το παιδί σας (μέσος όρος); - v',\
              'Πόσες ώρες περίπου παίζει βιντεοπαιχνίδια το παιδί σας τα Σαββατοκύριακα; - v', 'Πόσες φορές την εβδομάδα το παιδί σας είχε δυσκολίες στον ύπνο (π.χ. αϋπνία, δυσκολία στην έναρξη του ύπνου, συχνά ξυπνήματα, τρίξιμο δοντιών); - v',\
              'Πόσο συχνά χρησιμοποιείτε ελαιόλαδο στο μαγείρεμα; - v', 'Το παιδί μου ζητάει συνεχώς φαγητό - v', 'Πόσο συχνά τρώει το παιδί σας όσπρια; - v',\
              'Πόσο συχνά χρησιμοποιείτε βούτυρο; - v', 'Ακόμη κι αν έχει χορτάσει, το παιδί μου βρίσκει χώρο για το αγαπημένο του φαγητό - v', 'Πόσο συχνά χρησιμοποιείτε άλλα φυτικά έλαια στο μαγείρεμα; - v',\
              'Εάν είχε την επιλογή, το παιδί μου θα έτρωγε την περισσότερη ώρα της ημέρας - v', 'Πόσο συχνά τρώει το παιδί σας ξηρούς καρπούς (μια μερίδα αντιστοιχεί σε μια χούφτα ξηρών καρπών (π.χ. 18 αμύγδαλα, 6 ολόκληρα καρύδια, 3 κουταλιές της σούπας ηλιόσπορους)  ή προϊόντα τους  όπως ταχίνι, φυστικοβούτυρο κ.ά. (μια μερίδα αντιστοιχεί σε 1½  κουταλιά της σούπας των 15ml ή 25γρ. π.χ. ταχίνι) - v',\
              'Πόσο συχνά χρησιμοποιείτε μαργαρίνη; - v', 'Πόσες μερίδες φρούτων καταναλώνει το παιδί σας (μέσος όρος); - v', 'Πόσες μερίδες λαχανικών (ωμών ή βραστών ή στον ατμό ή ψητών) καταναλώνει το παιδί σας (μέσος όρος); - v',\
              'Πόσες μερίδες γαλακτοκομικών (ένα ποτήρι γάλα των 250ml, ένα κεσεδάκι γιαούρτι των 200γρ, μια φέτα τυρί του τοστ)  καταναλώνει  το παιδί σας (μέσος όρος); - v',\
              'Πόσες μερίδες δημητριακών καταναλώνει  το παιδί σας (μέσος όρος); - v', 'Από τις παραπάνω  μερίδες δημητριακών που καταναλώνει συνήθως το παιδί σας ημερησίως πόσες περιέχουν δημητριακά ολικής άλεσης όπως τοστ ολικής άλεσης, ψωμί ολικής άλεσης  ή πολύσπορο, παξιμάδια ολικής άλεσης, ζυμαρικά ολικής άλεσης, ποπ κορν, δημητριακά πρωινού με περιεκτικότητα σε φυτικές ίνες > 3γρ / 100γρ προϊόντος: - v',\
              'Πόσο συχνά  καταναλώνει το παιδί σας λευκό κρέας (κοτόπουλο, γαλοπούλα, πάπια, κουνέλι, κυνήγι); - v', 'Πόσο συχνά  καταναλώνει το παιδί σας κόκκινο κρέας (μοσχάρι, χοιρινό, κατσίκι, αρνί); - v',\
              'Πόσες φορές την εβδομάδα κατανάλωσε το παιδί σας επεξεργασμένο κρέας (π.χ. μια φέτα ζαμπόν, μια φέτα γαλοπούλα, μια φέτα σαλάμι, μια φέτα παριζάκι , ένα μεσαίο λουκάνικο) - v',\
              'Το παιδί μου έχει μεγάλη όρεξη - v', 'Πόσο συχνά τρώει το παιδί σας λαδερό φαγητό (π.χ. φασολάκια πράσινα, αρακά, μπριάμ); - v',\
              'Το παιδί μου ΔΕΝ χορταίνει εύκολα - v', 'Το παιδί μου ΔΕΝ αφήνει φαγητό στο πιάτο στο τέλος του γεύματος - v', 'Πόσο συχνά τρώει το παιδί σας έτοιμο φαγητό ταχυφαγείου (π.χ. πίτα με γύρο, πίτσα); - v',\
              'Το παιδί μου απορρίπτει νέα φαγητά στην αρχή - v', 'Πόσο συχνά καταναλώνει το παιδί σας αυγά (σε οποιαδήποτε μορφή, βραστά, τηγανητά, ομελέτα, σε συνταγές); - v', 'Το παιδί μου μπορεί να φάει ένα γεύμα αν έχει φάει σνακ λίγο πριν - v',\
              'Πόσο συχνά πίνει το παιδί σας  αναψυκτικά τύπου coca cola ή fanta (330ml)  ή χυμούς εμπορίου π.χ. αμίτα  (1 ποτήρι των 250 ml); - v', 'Το παιδί μου ΔΕΝ ενδιαφέρεται να δοκιμάσει νέα φαγητά - v',\
              'Πόσο συχνά τρώει το παιδί σας τυποποιημένα προϊόντα με πολλές θερμίδες (δηλαδή >250kcal ανά 100γρ. προϊόντος) και υψηλά σε απλά σάκχαρα (δηλαδή με σάκχαρα > 22,5γρ. ανά 100γρ προϊόντος) ή υψηλά σε αλάτι (δηλαδή > 1,5γρ. αλατιού ή 0,6γρ νατρίου ανά 100 γρ προϊόντος) (π.χ. 1 σακούλι πατατάκια 70γρ, 1 κρουασάν 100γρ, 1 ντόνατς 100γρ, 1 σοκολάτα 100γρ, ≥ 3 μπισκότα τύπου γεμιστά) - v',\
              'Το παιδί μου δύσκολα ικανοποιείται με τα γεύματα - v', 'ΟΙΚΟΓΕΝΕΙΑΚΑ ΓΕΥΜΑΤΑ - v', 'Τι είδους άσκηση έκανε το παιδί σας την προηγούμενη βδομάδα; - v', 'ΡΥΘΜΟΣ ΣΙΤΙΣΗΣ - v',\
              'Πόσα λεπτά την ημέρα περίπου ήταν το παιδί σας σωματικά δραστήριο (περπάτημα, ποδήλατο, πατίνι, παιχνίδια με φίλους που απαιτούν σωματική δραστηριότητα, κηπουρική, μαγειρική, δουλειές σπιτιού) - v',\
              'Πόσες φορές την εβδομάδα ασκήθηκε (οργανωμένο σπορ) το παιδί σας; - v', 'ΑΡΙΘΜΟΣ ΚΥΡΙΩΝ ΓΕΥΜΑΤΩΝ - v', 'Πόσες ώρες περίπου την ημέρα (καθημερινές)  πέρασε το παιδί σας  μπροστά σε οθόνες; (π.χ. τηλεόραση, τάμπλετ, κινητό) - v',\
              'Φύλο - v', 'Ηλικία - v', 'Με βάση το ύψος σας και το σωματικό σας βάρος, θεωρείτε πως είστε - v', 'Ποια είναι η οικογενειακή σας κατάσταση; - v',\
              'Ποια είναι η εθνικότητά σας*; - v', 'Ποιο είναι το μορφωτικό σας επίπεδο; - v',\
              'Ποια είναι η εργασία σας; (σε περίπτωση συνταξιοδότησης σημειώνετε την απασχόληση πριν από τη συνταξιοδότηση) - v', 'Είστε ικανοποιημένος με το εισόδημά σας; - v',\
              'Ποιο είναι το ακαθάριστο ετήσιο εισόδημά σας; (η ερώτηση αυτή είναι προαιρετική) - v', 'Είστε καπνιστής; - v', 'Το οικογενειακό σας εισόδημα καλύπτει τις ανάγκες σας; - v',\
              'Ασκείστε; Πόσο συχνά; - v', 'Καταναλώνετε αλκοολούχα ποτά; - v', 'Νοσείτε από κάποια χρόνια ασθένεια (πχ. Υπέρταση, Δυσλιπιδαιμία, Διαβήτη,); - v', 'Πόσο καλή είναι η υγεία σας; - v']

In [59]:
#replace with the most frequent value
for col in data_categ:
  data = data.fillna(data[col].value_counts().index[0])

In [60]:
for name,col_type in zip(data.columns,data.dtypes):
    if ("Score" in name):
      print (name,col_type)

Score - v float64
Score_psycho - v float64


## Get Dummies - One hot encoding

In [61]:
dum_catg_ftrs = ['target', 'home', 'junk', 'ΑΛΛΑ ΠΡΟΒΛΗΜΑΤΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v', 'ΝΟΣΗΜΑΤΑ ΜΗΤΡΟΣ - v', 'ΝΟΣΗΜΑΤΑ ΠΑΤΡΟΣ - v', 'ΚΥΗΣΗ - v', 'ΚΑΠΝΙΣΜΑ ΠΡΟ ΚΥΗΣΗΣ - v',\
                 'Σ.Δ. ΚΥΗΣΗΣ - v', 'IUGR (ενδομήτρια καθυστέρηση αύξησης) - v', 'ΚΑΠΝΙΣΜΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v', 'ΑΛΛΑ ΠΡΟΒΛΗΜΑΤΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v', 'SGA - v', 'ΤΟΚΕΤΟΣ - v',\
                 'ΙΚΤΕΡΟΣ - v', 'ΥΠΟΓΛΥΚΑΙΜΙΑ - v', 'ΠΑΡΑΜΟΝΗ ΣΕ ΜΕΝ / ΜΑΦ - v', 'ΘΗΛΑΣΜΟΣ - v', 'ΑΛΛΕΡΓΙΕΣ - v', 'ΦΑΡΜΑΚΑ / ΒΙΤΑΜΙΝΕΣ / ΣΥΜΠΛΗΡΩΜΑΤΑ - v', 'Χρόνια νοσήματα - v',\
                 'Συγγενή νοσήματα - v', 'ΝΟΣΗΛΕΙΕΣ - v', 'ΝΟΣΗΜΑΤΑ ΜΗΤΡΟΣ - v', 'ΝΟΣΗΜΑΤΑ ΠΑΤΡΟΣ - v', 'Τι είδους άσκηση έκανε το παιδί σας την προηγούμενη βδομάδα; - v',\
                 'Φύλο - v', 'Ποια είναι η οικογενειακή σας κατάσταση; - v', 'Ποια είναι η εθνικότητά σας*; - v', 'Ποιο είναι το μορφωτικό σας επίπεδο; - v',\
                 'Ποια είναι η εργασία σας; (σε περίπτωση συνταξιοδότησης σημειώνετε την απασχόληση πριν από τη συνταξιοδότηση) - v', 'Είστε καπνιστής; - v',\
                 'Νοσείτε από κάποια χρόνια ασθένεια (πχ. Υπέρταση, Δυσλιπιδαιμία, Διαβήτη,); - v']

data = pd.get_dummies(data, columns=dum_catg_ftrs)

# Create classes from steps feature

In [62]:
# 0 -5000 steps => Class 0 (No exercise)
# 5000 - 10000 steps => Class 1 (A little exercise)
# >=10000 steps => Class 2 (Lots of exercise)

data.loc[data["steps"] < 5000, "steps"] = 0
data.loc[((data["steps"] > 5000) & (data["steps"] < 10000)), "steps"] = 1
data.loc[(data["steps"] >= 10000), "steps"] = 2

In [63]:
print(data["steps"].unique())
print(data.groupby('steps').size())

[0 2 1]
steps
0    1762
1     384
2     503
dtype: int64


### Examine correlation between features

In [64]:
"""
plt.figure(figsize=(20, 8))
# Store heatmap object in a variable to easily access it when you want to include more features (such as title).
# Set the range of values to be displayed on the colormap from 0 to 1, and set the annotation to True to display the correlation values on the heatmap.
heatmap = sns.heatmap(data.corr(), vmin=0, vmax=1, annot=True)
# Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);
"""

"\nplt.figure(figsize=(20, 8))\n# Store heatmap object in a variable to easily access it when you want to include more features (such as title).\n# Set the range of values to be displayed on the colormap from 0 to 1, and set the annotation to True to display the correlation values on the heatmap.\nheatmap = sns.heatmap(data.corr(), vmin=0, vmax=1, annot=True)\n# Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.\nheatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);\n"

## Check again for missing values

In [65]:
#Print columns and their missing values after removing them
for column in data.columns:
    print(column,": ",data[column].isnull().sum())  

pid :  0
timestamp :  0
sleepAsleepMinutes :  0
distance :  0
steps :  0
weight :  0
HbA1c - v :  0
Chol - v :  0
HDL - v :  0
LDL - v :  0
TG - v :  0
Lp(a) - v :  0
SGOT - v :  0
SGPT - v :  0
γGT - v :  0
FSH - v :  0
TSH - v :  0
UA - v :  0
25 (OH) vitamin D - v :  0
FreeT4 - v :  0
Κορτιζόλη - v :  0
Ινσουλίνη - v :  0
Τεστοστερόνη - v :  0
ACTH - v :  0
LH - v :  0
Οιστραδιόλη - v :  0
Γλυκόζη - v :  0
ΗΛΙΚΙΑ ΜΗΤΡΟΣ - v :  0
ΒΑΡΟΣ ΜΗΤΡΟΣ - v :  0
ΥΨΟΣ ΜΗΤΡΟΣ - v :  0
ΗΛΙΚΙΑ ΠΑΤΡΟΣ - v :  0
ΒΑΡΟΣ ΠΑΤΡΟΣ - v :  0
ΥΨΟΣ ΠΑΤΡΟΣ - v :  0
ΑΔΕΛΦΙΑ - v :  0
Score - v :  0
ΜΕΣΟΔΙΑΣΤΗΜΑΤΑ ΜΕΤΑΞΥ ΓΕΥΜΑΤΩΝ: - v :  0
ΑΡΙΘΜΟΣ ΜΙΚΡΟΓΕΥΜΑΤΩΝ / ΣΝΑΚ - v :  0
Πόσες ώρες περίπου παίζει βιντεοπαιχνίδια  το παιδί σας τις καθημερινές; - v :  0
Πόσες ώρες περίπου την ημέρα (Σαββατοκύριακο ή αργίες)  πέρασε το παιδί σας μπροστά σε οθόνες; (π.χ. τηλεόραση, τάμπλετ, κινητό) - v :  0
Πόσο συχνά τρώει το παιδί σας ψάρια ή θαλασσινά; - v :  0
Πόσες ώρες την ημέρα περίπου κοιμάται το παιδί σας (μέσος όρος); -

### Remove duplicates

In [66]:
print(data.shape)
data = data.drop_duplicates()
print(data.shape)

(2649, 214)
(2647, 214)


## Remove string columns

In [67]:
data = data.select_dtypes(exclude=['object'])

## Split into train and test

In [68]:
X = data.loc[:, data.columns != 'steps']
print(X.columns)
X = X.drop(columns = ['sleepAsleepMinutes', 'distance', 'weight'])
y = data["steps"]

Index(['sleepAsleepMinutes', 'distance', 'weight', 'HbA1c - v', 'Chol - v',
       'HDL - v', 'LDL - v', 'TG - v', 'Lp(a) - v', 'SGOT - v',
       ...
       'Ποια είναι η εργασία σας; (σε περίπτωση συνταξιοδότησης σημειώνετε την απασχόληση πριν από τη συνταξιοδότηση) - v_3.0',
       'Ποια είναι η εργασία σας; (σε περίπτωση συνταξιοδότησης σημειώνετε την απασχόληση πριν από τη συνταξιοδότηση) - v_4.0',
       'Ποια είναι η εργασία σας; (σε περίπτωση συνταξιοδότησης σημειώνετε την απασχόληση πριν από τη συνταξιοδότηση) - v_5.0',
       'Ποια είναι η εργασία σας; (σε περίπτωση συνταξιοδότησης σημειώνετε την απασχόληση πριν από τη συνταξιοδότηση) - v_8.0',
       'Ποια είναι η εργασία σας; (σε περίπτωση συνταξιοδότησης σημειώνετε την απασχόληση πριν από τη συνταξιοδότηση) - v_9.0',
       'Είστε καπνιστής; - v_0.0', 'Είστε καπνιστής; - v_1.0',
       'Είστε καπνιστής; - v_2.0',
       'Νοσείτε από κάποια χρόνια ασθένεια (πχ. Υπέρταση, Δυσλιπιδαιμία, Διαβήτη,); - v_0.0',
       'Νοσείτε α

In [69]:
#data = data.drop(columns=['pid', 'timestamp'])

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify = y, test_size = 0.30) # random_state = 42
X_train = X_train.reset_index()
X_test = X_test.reset_index()

### Training

In [70]:
# Parameters
# PCA
numOfFeatures = X_train.shape[1]
pca_components = [numOfFeatures,int(numOfFeatures/2),int(numOfFeatures/3)]
# SVM
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
gamma = ['scale','auto']
# KNN
n_neighbors = [5,10,15]
# Adaboost
learning_rate = [1e-4,1e-2,1e-1]
n_estimators = [50,100,150]
# RandomForest
criterion = ['gini','entropy']
max_features = ['auto',15,5,None]
min_samples_leaf = [20,50]
# Logistic Regression
penalty = ['l2']
solver = ['newton-cg', 'lbfgs', 'liblinear','sag', 'saga'] 
C = [0.5,1.0]
# Decision Tree
splitter = ['best','random']
# MLP Classifer
hidden_layer_sizes = [(100,),(50,),(150,)]
activation = ['logistic', 'tanh', 'relu']
solver = ['lbfgs','adam']
max_iter = [400,500,1000]
# Gradient Boosting
criterion1 = ['friedman_mse']
#  LinearDiscriminantAnalysis
solver1 = ['svd', 'lsqr', 'eigen']
#  XGBoost
objective = ["multi:softprob"]

Parameters = [dict(pca__n_components=pca_components,clf__objective = objective),
              dict(pca__n_components=pca_components,clf__kernel = kernel,clf__gamma = gamma),
              dict(pca__n_components=pca_components,clf__n_neighbors = n_neighbors),
              dict(pca__n_components=pca_components,clf__learning_rate = learning_rate,clf__n_estimators = n_estimators),
              dict(pca__n_components=pca_components,clf__n_estimators = n_estimators,clf__criterion = criterion,clf__max_features = max_features,clf__min_samples_leaf = min_samples_leaf),
              dict(pca__n_components=pca_components,clf__penalty = penalty,clf__C = C,clf__solver = solver),
              dict(pca__n_components=pca_components,clf__criterion = criterion,clf__splitter = splitter),
              dict(pca__n_components=pca_components),
              dict(pca__n_components=pca_components,clf__max_iter = max_iter,clf__hidden_layer_sizes = hidden_layer_sizes,clf__activation = activation,clf__solver = solver),
              dict(pca__n_components=pca_components,clf__learning_rate = learning_rate,clf__n_estimators = n_estimators,clf__criterion = criterion1),
              dict(pca__n_components=pca_components,clf__solver = solver1)]

"""
# Non PCA
Parameters = [dict(clf__objective = objective),
              dict(clf__kernel = kernel,clf__gamma = gamma),
              dict(clf__n_neighbors = n_neighbors),
              dict(clf__learning_rate = learning_rate,clf__n_estimators = n_estimators),
              dict(clf__n_estimators = n_estimators,clf__criterion = criterion,clf__max_features = max_features,clf__min_samples_leaf = min_samples_leaf),
              dict(clf__penalty = penalty,clf__C = C,clf__solver = solver),
              dict(clf__criterion = criterion,clf__splitter = splitter),
              dict(),
              dict(clf__max_iter = max_iter,clf__hidden_layer_sizes = hidden_layer_sizes,clf__activation = activation,clf__solver = solver),
              dict(pca__n_components=pca_components,clf__learning_rate = learning_rate,clf__n_estimators = n_estimators,clf__criterion = criterion1),
              dict(clf__solver = solver1)]
"""

# xgb.XGBRegressor(),

Classifiers = [xgb.XGBClassifier(),
               SVC(),
               KNeighborsClassifier(),
               AdaBoostClassifier(),
               RandomForestClassifier(),
               LogisticRegression(),
               DecisionTreeClassifier(),
               GaussianNB(),
               MLPClassifier(),
               GradientBoostingClassifier(),
               LinearDiscriminantAnalysis()]

for i,clf in enumerate(Classifiers):
    
    print(clf.__class__.__name__)
    # αρχικοποιούμε τον εκτιμητή (ταξινομητής) και τους μετασχηματιστές χωρίς υπερ-παραμέτρους
    selector = VarianceThreshold()
    rus = RandomUnderSampler()
    ros = RandomOverSampler()
    scaler = StandardScaler()    
    pca = PCA()    
    
    #pipe = Pipeline(steps=[('selector', selector),('scaler', scaler),('pca', pca), ('clf', clf)], memory = 'tmp')
    pipe = Pipeline(steps=[('selector', selector),('scaler', scaler),('rus', rus),('pca', pca), ('clf', clf)], memory = 'tmp')
    #pipe = Pipeline(steps=[('selector', selector),('scaler', scaler),('ros', ros),('pca', pca), ('clf', clf)], memory = 'tmp')
    # selector__threshold=vthreshold,
    estimator = GridSearchCV(pipe, param_grid = Parameters[i], cv=10, scoring='f1_macro')
    estimator.fit(X_train, y_train)
    print(estimator.best_estimator_)
    print(estimator.best_params_)
    
    preds = estimator.predict(X_test)
    print(classification_report(y_test, preds))

MLPClassifier
Pipeline(memory='tmp',
         steps=[('selector', VarianceThreshold()), ('scaler', StandardScaler()),
                ('rus', RandomUnderSampler()), ('pca', PCA(n_components=104)),
                ('clf',
                 MLPClassifier(activation='logistic', hidden_layer_sizes=(150,),
                               max_iter=400, solver='lbfgs'))])
{'clf__activation': 'logistic', 'clf__hidden_layer_sizes': (150,), 'clf__max_iter': 400, 'clf__solver': 'lbfgs', 'pca__n_components': 104}
              precision    recall  f1-score   support

           0       0.90      0.69      0.78       529
           1       0.25      0.42      0.31       115
           2       0.41      0.53      0.46       151

    accuracy                           0.62       795
   macro avg       0.52      0.55      0.52       795
weighted avg       0.71      0.62      0.65       795

GradientBoostingClassifier
Pipeline(memory='tmp',
         steps=[('selector', VarianceThreshold()), ('scaler', St